In [ ]:
import scanpy as sc
import pandas as pd
import sys
import os
import warnings
from gprofiler import GProfiler
warnings.filterwarnings("ignore")

#import gseapy

In [ ]:
adata = sc.read('../../data/all11sets_hm.h5')

### DE GENES

In [ ]:
studies = ['Lee et al.', 'Wilk et al.', 'Li et al.', 'Yu et al.', 'Zhang et al.', 'Wen et al.', 'Liao et al.', 'Chua et al.']
for currentStudy in studies:
    temp = adata[adata.obs.study==currentStudy]
    print(currentStudy)
    cell_types = [x for x in temp.obs.cell_type.unique().to_list() if x not in ['doublets','Epithelial','Megakaryocytes','Platelets']]
    for currentCell in cell_types:
        print(currentCell)
        sub = temp[temp.obs.cell_type==currentCell]
        try:
            sc.tl.rank_genes_groups(sub, 'stage', method='wilcoxon', n_genes=5000)
            result = sub.uns['rank_genes_groups']
            groups = result['names'].dtype.names
            df = pd.DataFrame({group + '_' + key[:1]: result[key][group] for group in groups for key in ['names', 'logfoldchanges', 'pvals_adj']})
            df.to_csv('../../dePathwayAndOtherAnalysisInR/results/DE/DE_'+currentStudy+'_'+currentCell+'.csv')
        except:
            pass

### The following R-code selects the common DE genes across studies for each stage, for each cell-type

In [ ]:
### R code ###
### Select common DE across studies for each stage, for each cell-type
# allFiles <- list.files('../../dePathwayAndOtherAnalysisInR/results/DE/', pattern = "DE_.*", full.names = TRUE)
# df <- data.frame()
# for(file in allFiles){
#     temp<-read.csv(file)
#     temp<-temp[, -1]
#     divCols<-ncol(temp)
#     temp1<-data.frame()
#     for(n in 1:(divCols/3)){
#         sub<-temp[, ((3*n)-2):(3*n)]
#         sub$stage<-unique(unlist(strsplit(names(sub), split="_")))[1]
#         names(sub)<-c("gene", "lfc", "padj", "stage")
#         temp1<-rbind(sub, temp1)
#     }
#         title<-unlist(strsplit(tools::file_path_sans_ext(basename(file)), split="_"))
#         temp1$study<-title[2]
#         temp1$cell_type<-title[3]
#     df<-rbind(temp1, df)
#     }

# #Keep differentially expressed genes only
# df_de<-df[df$padj<0.05, ]
# df_de<-df_de[-c(grep("^RPS", df_de$gene), grep("^RPL", df_de$gene), 
#                 grep("^MT-", df_de$gene), grep("^MTR", df_de$gene),
#                 grep("^RNA1\\dS5", df_de$gene)),]

# common_genes_in_all_studies<-data.frame()
# for(currentStage in unique(df_de$stage)){
#     for(currentCell in unique(df_de$cell_type)){
#         sub<-droplevels(df_de[(df_de$stage==currentStage) & (df_de$cell_type==currentCell) &(!df_de$study%in%c('Chua et al.', 'Liao et al.')), ])
#         freq_table<-data.frame(table(sub$gene))
#         #interested only in genes that occur in all the studies having that stage
#         common_genes_in_all_studies<-rbind(sub[sub$gene %in% 
#                                                freq_table$Var1[freq_table$Freq==length(unique(sub$study))], ],
#                                            common_genes_in_all_studies)
        
#     }
# }
# write.table(common_genes_in_all_studies, "../../dePathwayAndOtherAnalysisInR/results/DE/common_genes_in_all_studies_chua_liao_excluded.csv",
#            sep=",", col.names=TRUE, row.names=TRUE, quote=FALSE)

## Pathway analysis

In [ ]:
df = pd.read_csv("../../dePathwayAndOtherAnalysisInR/results/DE/common_genes_in_all_studies_chua_liao_excluded.csv")

for index, row in df.loc[:, ['stage', 'cell_type']].drop_duplicates().iterrows():
    markers = df[(df.cell_type==row['cell_type'])&(df.stage==row['stage'])].gene.unique().tolist()
    gp = GProfiler(return_dataframe=True, user_agent='g:GOSt')
    enrichmentResults = gp.profile(organism='hsapiens', user_threshold=0.05,
                               significance_threshold_method='g_SCS', 
                               background=adata.var_names.tolist(), 
                               ordered=True, no_iea=True,
                               query=markers)
    outEnrich='../../dePathwayAndOtherAnalysisInR/results/DE/GProfiler_'+row['cell_type']+'_'+row['stage']+'.tsv'
    enrichmentResults.to_csv(outEnrich, sep = "\t")

### Plotting pathway analysis results

In [1]:
from gprofiler_plotting import plot_enrich
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10]
pd.set_option("display.max_colwidth", 800)
group='GProfiler_CD14 monocytes_Severe'
pathways = pd.read_csv('../../dePathwayAndOtherAnalysisInR/results/DE/'+group+'.tsv', sep="\t")
#pathways[pathways.source=='GO:BP'].name.unique().tolist()
enrich_results = pathways[pathways.source=='GO:BP'].set_index('native').sort_values('p_value').loc[:,['p_value','term_size','intersection_size','recall','name']]
enrich_results.iloc[:50,:]
plot_enrich(enrich_results, n_terms=30, save='../../dePathwayAndOtherAnalysisInR/results/DE/'+group+'.pdf')

ModuleNotFoundError: No module named 'gprofiler_plotting'